In [1]:
import torch
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Download the FashionMNIST dataset
train_data = datasets.FashionMNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.FashionMNIST(root='data', train=False,
                                  download=True, transform=transform)

# Remove even classes from the dataset
odd_classes = [1,3,5,7,9]

odd_idx_train = [i for i in range(len(train_data)) if train_data[i][1] in odd_classes]
odd_idx_test = [i for i in range(len(test_data)) if test_data[i][1] in odd_classes]

mnist_subset_train = torch.utils.data.Subset(train_data, odd_idx_train)
mnist_subset_test = torch.utils.data.Subset(test_data, odd_idx_test)

a = torch.zeros([len(mnist_subset_train),1,28,28])
b = torch.zeros([len(mnist_subset_train)])
for i, (image, label) in enumerate(mnist_subset_train):
    a[i] = image
    if label == 1:
        b[i] = int(0)
    elif label == 3:
        b[i] = int(1)
    elif label == 5:
        b[i] = int(2)
    elif label == 7:
        b[i] = int(3)
    elif label == 9:
        b[i] = int(4)
train_dataset = torch.utils.data.TensorDataset(a,b)

p = torch.zeros([len(mnist_subset_test),1,28,28])
q = torch.zeros([len(mnist_subset_test)])
for i, (image, label) in enumerate(mnist_subset_test):
    p[i] = image
    if label == 1:
        q[i] = int(0)
    elif label == 3:
        q[i] = int(1)
    elif label == 5:
        q[i] = int(2)
    elif label == 7:
        q[i] = int(3)
    elif label == 9:
        q[i] = int(4)
test_dataset = torch.utils.data.TensorDataset(p,q)

# Create dataloaders for train and test sets
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


100%|██████████| 26421880/26421880 [00:00<00:00, 108187359.33it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 5956832.00it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




100%|██████████| 4422102/4422102 [00:00<00:00, 62619482.06it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 19900716.12it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [2]:
print(len(train_dataset))
print(len(test_dataset))

30000
5000


In [3]:
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models

In [4]:
model= models.resnet18( num_classes=5)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)
#model.fc = torch.nn.Linear(512 ,10)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.1, amsgrad=False)

In [7]:
def train(model, train_loader, criterion, optimizer, device):
    # Set the model to training mode
    model.train()

    # Initialize variables to keep track of training loss and accuracy
    train_loss = 0.0
    train_acc = 0.0

    # Process the images in batches
    for batch_idx, (data, target) in enumerate(train_loader):
        # Move the data to the device
        data, target = data.to(device), target.to(device).long()

        # Reset the optimizer
        optimizer.zero_grad()

        # Push the data forward through the model layers
        output = model(data)

        # Get the loss
        loss = criterion(output, target)

        # Compute gradients and backpropagate
        loss.backward()
        optimizer.step()

        # Update the training loss and accuracy
        train_loss += loss.item() * data.size(0)
        _, predicted = torch.max(output, 1)
        train_acc += (predicted == target).sum().item()

        # Print progress every 100 batches
        if (batch_idx + 1) % 100 == 0:
            print('Batch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(batch_idx + 1, len(train_loader), train_loss / ((batch_idx + 1) * data.size(0)),
                          100.0 * train_acc / ((batch_idx + 1) * data.size(0))))

    # Compute the average training loss and accuracy for the epoch
    avg_train_loss = train_loss / len(train_loader.dataset)
    avg_train_acc = 100.0 * train_acc / len(train_loader.dataset)

    return avg_train_loss, avg_train_acc


In [8]:
for epoch in range(30):
    # Train the model for one epoch
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)

Batch [100/469], Loss: 0.3112, Accuracy: 89.59%
Batch [200/469], Loss: 0.2906, Accuracy: 90.29%
Batch [300/469], Loss: 0.2816, Accuracy: 90.57%
Batch [400/469], Loss: 0.2793, Accuracy: 90.75%
Batch [100/469], Loss: 0.2564, Accuracy: 92.09%
Batch [200/469], Loss: 0.2465, Accuracy: 92.38%
Batch [300/469], Loss: 0.2516, Accuracy: 92.26%
Batch [400/469], Loss: 0.2518, Accuracy: 92.24%
Batch [100/469], Loss: 0.2640, Accuracy: 91.80%
Batch [200/469], Loss: 0.2540, Accuracy: 92.38%
Batch [300/469], Loss: 0.2575, Accuracy: 92.32%
Batch [400/469], Loss: 0.2544, Accuracy: 92.61%
Batch [100/469], Loss: 0.2634, Accuracy: 93.56%
Batch [200/469], Loss: 0.2682, Accuracy: 93.66%
Batch [300/469], Loss: 0.2750, Accuracy: 93.72%
Batch [400/469], Loss: 0.2782, Accuracy: 93.91%
Batch [100/469], Loss: 0.3033, Accuracy: 94.94%
Batch [200/469], Loss: 0.3008, Accuracy: 94.97%
Batch [300/469], Loss: 0.3023, Accuracy: 94.95%
Batch [400/469], Loss: 0.3007, Accuracy: 95.00%
Batch [100/469], Loss: 0.2783, Accuracy:

In [9]:
def test(model, test_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0.0
    with torch.no_grad():  # Disable gradient tracking
        for batch_idx, (data, target) in enumerate(test_loader):
            # Move the data to the device
            data, target = data.to(device), target.to(device).long()
            output = model(data)  # Push the data forward through the model layers
            test_loss = criterion(output, target)  # Get the loss
            test_loss += test_loss.item() * data.size(0)
            _, pred = torch.max(output, 1)
            #pred = output.argmax(dim=1, keepdim=True)  # Get the index of the highest probability
            correct += pred.eq(target.view_as(pred)).sum().item()  # Update the number of correct predictions

    # Print the average loss and accuracy for the test set
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print('Test set | Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))
    return test_loss, accuracy

In [10]:
#for epoch in range(30):
test_loss, test_acc = test(model, test_loader, criterion)

Test set | Average loss: 0.0006, Accuracy: 4823.0/5000 (96.46%)


In [11]:
%pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 27.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=9387a3db192b729255ee87535db6d4ddc1a0a885c727973e5a9c8c8adfc8cacc
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [12]:
import wandb 
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [13]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="FashionMNIST Project",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.001,
    "architecture": "ResNet18",
    "dataset": "FashionMNIST",
    "epochs": 30,
    }
)

# simulate training
epochs = 30
for epoch in range(epochs):
    # log metrics to wandb
    wandb.log({"acc": train_acc, "loss": train_loss})
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

wandb: Currently logged in as: princyg561 (ssup_a). Use `wandb login --relogin` to force relogin


acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,96.34
loss,0.24724


In [14]:
class ResNet18_model2(nn.Module):
    def __init__(self, num_classes=5):
        super(ResNet18_model2, self).__init__()
        self.resnet18 = models.resnet18(pretrained=True)
        self.resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)
        self.resnet18.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = F.tanh(self.resnet18(x))
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
Model2 = ResNet18_model2()
Model2.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_sgd = optim.SGD(Model2.parameters(), lr=0.01, momentum=0.9)

for epoch in range(30):
    # Train the model for 30 epoch
    train_loss_Model2, train_acc_Model2 = train(Model2, train_loader, criterion, optimizer_sgd, device)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 148MB/s]


Batch [100/469], Loss: 0.6592, Accuracy: 86.55%
Batch [200/469], Loss: 0.6082, Accuracy: 89.36%
Batch [300/469], Loss: 0.5876, Accuracy: 90.65%
Batch [400/469], Loss: 0.5740, Accuracy: 91.36%
Batch [100/469], Loss: 0.5097, Accuracy: 95.38%
Batch [200/469], Loss: 0.5132, Accuracy: 95.23%
Batch [300/469], Loss: 0.5138, Accuracy: 95.06%
Batch [400/469], Loss: 0.5110, Accuracy: 95.21%
Batch [100/469], Loss: 0.4867, Accuracy: 96.69%
Batch [200/469], Loss: 0.4921, Accuracy: 96.48%
Batch [300/469], Loss: 0.4905, Accuracy: 96.56%
Batch [400/469], Loss: 0.4891, Accuracy: 96.64%
Batch [100/469], Loss: 0.4788, Accuracy: 97.20%
Batch [200/469], Loss: 0.4800, Accuracy: 97.06%
Batch [300/469], Loss: 0.4801, Accuracy: 97.10%
Batch [400/469], Loss: 0.4796, Accuracy: 97.13%
Batch [100/469], Loss: 0.4765, Accuracy: 97.31%
Batch [200/469], Loss: 0.4741, Accuracy: 97.41%
Batch [300/469], Loss: 0.4728, Accuracy: 97.53%
Batch [400/469], Loss: 0.4730, Accuracy: 97.48%
Batch [100/469], Loss: 0.4657, Accuracy:

In [15]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="FashionMNIST Project 2",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.001,
    "architecture": "ResNet18",
    "dataset": "FashionMNIST",
    "epochs": 30,
    }
)

# simulate training
epochs = 30
for epoch in range(epochs):
    # log metrics to wandb
    wandb.log({"acc": train_acc_Model2, "loss": train_loss_Model2})
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,99.47667
loss,0.44176
